# HexaSudoku Board Image Generation

Generate 1600×1600px PNG images from 16×16 Sudoku puzzle JSON data.

Key features:
- 1600×1600 pixels (100px per cell)
- Bold lines for 4×4 box boundaries
- Hexadecimal notation: 1-9 and A-F for values 10-15
- Numbers centered in cells
- Empty cells shown as blank (0 in puzzle = empty)

In [ ]:
import json
import os

with open("./puzzles/puzzles_dict.json", "r") as f:
    puzzles_ds = json.load(f)

print(f"Loaded puzzles: {list(puzzles_ds.keys())}")
print(f"Number of 16x16 puzzles: {len(puzzles_ds.get('16', []))}")

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt

## Constants

In [ ]:
SIZE = 16
BOX_SIZE = 4
BOARD_PIXELS = 1600
CELL_SIZE = BOARD_PIXELS // SIZE  # 100px per cell

## Helper Functions

In [ ]:
def value_to_char(value):
    """
    Convert puzzle value (0-16) to display character.
    0 = empty, 1-9 = digits, 10-15 = A-F
    """
    if value == 0:
        return ''
    if value <= 9:
        return str(value)
    return chr(ord('A') + value - 10)  # 10→A, 11→B, ..., 15→F

## Board Generation Functions

In [ ]:
def make_hexasudoku_board():
    """
    Create a blank 16x16 Sudoku board with grid lines.
    Bold lines for 4x4 box boundaries.
    
    Returns:
        1600x1600 numpy array (1.0 = white, 0.0 = black)
    """
    grid = np.ones((BOARD_PIXELS, BOARD_PIXELS))
    
    # Draw outer border (bold)
    border_thickness = 8
    for i in range(border_thickness):
        grid[i, :] = 0
        grid[-i-1, :] = 0
        grid[:, i] = 0
        grid[:, -i-1] = 0
    
    # Draw internal grid lines
    for i in range(1, SIZE):
        pos = i * CELL_SIZE
        
        # Bold lines for box boundaries (every 4 cells)
        if i % BOX_SIZE == 0:
            thickness = 6
        else:
            thickness = 2
        
        for j in range(-thickness, thickness):
            if 0 <= pos + j < BOARD_PIXELS:
                grid[pos + j, :] = 0  # Horizontal line
                grid[:, pos + j] = 0  # Vertical line
    
    return grid

In [ ]:
def insert_character(img, row, col, value):
    """
    Insert a character (digit or letter) into a cell using PIL text rendering.
    Character is centered in the cell.
    
    Args:
        img: PIL Image
        row, col: Cell position
        value: Puzzle value (0-15)
    
    Returns:
        Modified PIL Image
    """
    if value == 0:
        return img  # Empty cell
    
    char = value_to_char(value)
    draw = ImageDraw.Draw(img)
    
    # Try to use a nice font, fall back to default
    try:
        font_size = CELL_SIZE * 2 // 3  # ~67px for 100px cells
        font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", font_size)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", font_size)
        except:
            font = ImageFont.load_default()
    
    # Calculate centered position
    cell_center_x = col * CELL_SIZE + CELL_SIZE // 2
    cell_center_y = row * CELL_SIZE + CELL_SIZE // 2
    
    bbox = draw.textbbox((0, 0), char, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    x = cell_center_x - text_width // 2
    y = cell_center_y - text_height // 2 - bbox[1]  # Adjust for baseline
    
    draw.text((x, y), char, fill=0, font=font)
    
    return img

In [ ]:
def make_board_full(puzzle):
    """
    Create a complete HexaSudoku board image with given clues.
    
    Args:
        puzzle: 16x16 2D list where 0 = empty cell, 1-16 = filled
    
    Returns:
        1600x1600 numpy array
    """
    # Create base grid
    grid = make_hexasudoku_board()
    
    # Convert to PIL Image for text rendering
    img = Image.fromarray((grid * 255).astype(np.uint8), mode='L')
    
    # Insert all characters
    for row in range(SIZE):
        for col in range(SIZE):
            value = puzzle[row][col]
            if value != 0:
                img = insert_character(img, row, col, value)
    
    return np.array(img) / 255.0

In [ ]:
def make_and_save(puzzle, idx):
    """
    Generate and save a HexaSudoku board image.
    """
    board = make_board_full(puzzle)
    array_uint8 = (board * 255).astype(np.uint8)
    image = Image.fromarray(array_uint8, mode='L')
    
    output_dir = './board_images'
    os.makedirs(output_dir, exist_ok=True)
    
    filename = f"{output_dir}/board16_{idx}.png"
    image.save(filename)
    return filename

## Demo: Generate Single Board

In [ ]:
# Demo with first puzzle
if '16' in puzzles_ds and len(puzzles_ds['16']) > 0:
    puzzle = puzzles_ds['16'][0]['puzzle']
    board = make_board_full(puzzle)
    
    # Count clues
    clues = sum(1 for row in puzzle for cell in row if cell != 0)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(board, cmap='gray')
    plt.title(f"16×16 HexaSudoku Puzzle ({clues} clues)")
    plt.axis('off')
    plt.show()
else:
    print("No puzzles loaded. Run SymbolicPuzzleGenerator.ipynb first.")

## Demo: Show Hex Character Range

In [ ]:
# Create a demo board showing all 16 values
demo_puzzle = [[0]*16 for _ in range(16)]

# Fill first row with all values 1-16
for i in range(16):
    demo_puzzle[0][i] = i + 1  # 1-16

demo_board = make_board_full(demo_puzzle)

plt.figure(figsize=(12, 3))
plt.imshow(demo_board[:120, :], cmap='gray')  # Just show top portion
plt.title("All 16 values: 1 2 3 4 5 6 7 8 9 A B C D E F (10-15)")
plt.axis('off')
plt.show()

print("Value mapping:")
for i in range(1, 17):
    print(f"  {i:2d} → {value_to_char(i)}")

## Generate All Board Images

In [ ]:
# Generate images for all puzzles
if '16' in puzzles_ds:
    puzzles = puzzles_ds['16']
    
    print(f"Generating {len(puzzles)} images for 16×16 HexaSudoku puzzles...")
    
    for i, puzzle_data in enumerate(puzzles):
        puzzle = puzzle_data['puzzle']
        make_and_save(puzzle, i)
        
        if (i + 1) % 25 == 0:
            print(f"  Generated {i + 1}/{len(puzzles)}")
    
    print(f"\nTotal images generated: {len(puzzles)}")
else:
    print("No puzzles loaded. Run SymbolicPuzzleGenerator.ipynb first.")

## Verify Generated Images

In [ ]:
# List generated images
import glob

images = glob.glob('./board_images/*.png')
print(f"Total images in board_images: {len(images)}")

In [ ]:
# Display a few random examples
import random

if len(images) > 0:
    sample_images = random.sample(images, min(3, len(images)))
    
    fig, axes = plt.subplots(1, len(sample_images), figsize=(6*len(sample_images), 6))
    if len(sample_images) == 1:
        axes = [axes]
    
    for ax, img_path in zip(axes, sample_images):
        img = Image.open(img_path)
        ax.imshow(img, cmap='gray')
        ax.set_title(os.path.basename(img_path))
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No images generated yet.")